In [1]:
## this file buys and sells on jupiter
### keep in mind the amount is in lamports so 1 sol would be 1,000,000,000
# to run the code: python jup.py 8wXtPeU6557ETkp9WHFY1n1EcU6NxDvbAggHGsMYiHsB 100


########################################################

import requests
import sys
import json 
import base64
from solders.keypair import Keypair
from solders.transaction import VersionedTransaction 
from solana.rpc.api import Client
from solana.publickey import PublicKey 
from solana.rpc.types import TxOpts
from solana.rpc.types import TokenAccountOpts
import dontshare as d 
from pprint import pprint


In [11]:
def get_decimals(token_mint_address):
    import requests
    import base64
    import json
    # Solana Mainnet RPC endpoint
    url = "https://api.mainnet-beta.solana.com/"
    headers = {"Content-Type": "application/json"}

    # Request payload to fetch account information
    payload = json.dumps({
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getAccountInfo",
        "params": [
            token_mint_address,
            {
                "encoding": "jsonParsed"
            }
        ]
    })

    # Make the request to Solana RPC
    response = requests.post(url, headers=headers, data=payload)
    response_json = response.json()

    # Parse the response to extract the number of decimals
    decimals = response_json['result']['value']['data']['parsed']['info']['decimals']
    #print(f"Decimals for {token_mint_address[-4:]} token: {decimals}")

    return decimals

token_decimals = get_decimals('So11111111111111111111111111111111111111112')
print(token_decimals)

def token_price(address):
    API_KEY = d.birdeye
    url = f"https://public-api.birdeye.so/defi/price?address={address}"
    headers = {"X-API-KEY": API_KEY}
    response = requests.get(url, headers=headers)
    price_data = response.json()
    
    if price_data['success']:
        return price_data['data']['value']
    else:
        return None

token_price = token_price('So11111111111111111111111111111111111111112')    
print(token_price)

9
153.60085886036507


In [12]:
## 1 SOL = 1,000,000,000 lamports 
## Decimal = 9 = is the number of zeros after the 1 in the token price
## SOL price = token_price = 153.60 USD
## 1 USD = 1/SOL price = 1/153.60 SOL
## Order Quantity Lamports of Token = (USDT_Quantity / Token_Price) * 10^(token_decimals) 

In [2]:
# KEY FROM THE FILE DONTSHARE
KEY = Keypair.from_base58_string(d.key)
SLIPPAGE = 100 # 50 bps = 0.5%

QUOTE_TOKEN = 'So11111111111111111111111111111111111111112' # Token we are swapping from

token = 'EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm' #sys.argv[1]

#QUOTE_TOKEN = 'EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm' # Token we are swapping from

#token = 'So11111111111111111111111111111111111111112' #sys.argv[1]

amount = 1000000 #sys.argv[2]
#amount = 56252
# 1000000/1,000,000,000 = 0.001 SOL = 0.001 * 156.6 = 0.1566 USD


http_client = Client("https://api.mainnet-beta.solana.com")

quote = requests.get(f'https://quote-api.jup.ag/v6/quote?inputMint={QUOTE_TOKEN}\
&outputMint={token}\
&amount={amount}\
&slippageBps={SLIPPAGE}').json()
pprint(quote)



{'contextSlot': 295847484,
 'inAmount': '1000000',
 'inputMint': 'So11111111111111111111111111111111111111112',
 'otherAmountThreshold': '58231',
 'outAmount': '58819',
 'outputMint': 'EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm',
 'platformFee': None,
 'priceImpactPct': '0',
 'routePlan': [{'percent': 100,
                'swapInfo': {'ammKey': '5XRqv7LCoC5FhWKk5JN8n4kCrJs3e4KH1XsYzKeMd5Nt',
                             'feeAmount': '500',
                             'feeMint': 'So11111111111111111111111111111111111111112',
                             'inAmount': '1000000',
                             'inputMint': 'So11111111111111111111111111111111111111112',
                             'label': 'Meteora DLMM',
                             'outAmount': '154766',
                             'outputMint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v'}},
               {'percent': 100,
                'swapInfo': {'ammKey': '2gwEFGbu7KVgocNnmPa7JgZEuerf4WxTUFzTEJeU4U4b',
        

### Fields Breakdown: Quote Response

#### inputMint

The mint address of the input token. In your case, it's `So11111111111111111111111111111111111111112` (SOL).

#### inAmount

The amount of the input token being swapped, in lamports (1 SOL = 1,000,000,000 lamports). Here, `1000000` lamports = 0.001 SOL.

#### outputMint

The mint address of the output token, which matches the token variable printed earlier.

#### outAmount

The expected amount of output tokens you'll receive. Here, `55899` units of the output token.

#### otherAmountThreshold

The minimum acceptable amount of output tokens to receive, accounting for slippage.

#### swapMode

Indicates the type of swap operation. `ExactIn` means you're specifying the exact input amount.

#### slippageBps

Slippage tolerance in basis points. `50` bps = 0.5%.

#### platformFee

Any additional fees by the platform. `null` indicates no extra platform fee.

#### priceImpactPct

The percentage of price impact due to the swap. `0` indicates minimal or no price impact.

#### routePlan

An array detailing each step of the swap across different Automated Market Makers (AMMs).

##### Each swapInfo Object

- **ammKey:** Unique identifier for the AMM.
- **label:** Name of the AMM.
- **inputMint & outputMint:** Mint addresses for tokens being swapped.
- **inAmount & outAmount:** Amounts being swapped in and out.
- **feeAmount & feeMint:** Fees associated with the swap.
- **percent:** The percentage split for this swap route (`100%` in each case here, indicating a sequential swap).

#### contextSlot

The Solana blockchain slot at which this quote was generated.

#### timeTaken

Time taken to generate this quote, measured in seconds.

In [3]:

txRes = requests.post('https://quote-api.jup.ag/v6/swap',headers={"Content-Type": "application/json"}, data=json.dumps({"quoteResponse": quote, "userPublicKey": str(KEY.pubkey()) })).json()
pprint(txRes)



{'computeUnitLimit': 1400000,
 'dynamicSlippageReport': None,
 'lastValidBlockHeight': 274572063,
 'prioritizationFeeLamports': 99999,
 'prioritizationType': {'computeBudget': {'estimatedMicroLamports': 1857194,
                                          'microLamports': 71428}},
 'simulationError': None,
 'swapTransaction': 'AQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAQAHDyi0vkzkwucylsdvIg2yi0IC2ruMDVOUZbhJMtUToxDqbcs4CQAHfNq+4JE5TMh5kCw6ImH8caqpjjlmgNrAV0KWBZPRhBharueMiBQvSlJTy+U0TnhT5rLpalSEc0WK+qIdmklnVTL3GrXKI61rcFwE4jsO/6OYROzRbst6fL4CuCsOTPx5nwfa6qYwEbOhdQRY+2kwI9+FtDWWtY5Z1su+BTzempcQw503kebhNy2MK8YVMV4BB2Aq6Sr6a782C8KUY8q4L9omMMPzB4Ctj/RWVHXqRQovWPTCLeBrMXYQ37C4X+xX/TaH3D4CMgGzMrZuYIE8isJkfiaoF1FwEVUAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAMGRm/lIRcy/+ytunLDm+e8jOW7xfcSayxDmzpAAAAABHnVW/IxwG7udMVuzmgVB/2xst6j9I5RArHNola8E48G3fbh12Whk9nL4UbO63msHLSF7V9bN5E6jPWFfv8AqU9LbA5BCP0qaiR46uCsxZ2yG7Tt9RHBYs9gLR4MCQH7jJclj04kifG7PRApFI4NgwtaE

### Fields Breakdown: Swap Transaction

#### swapTransaction

A base64-encoded string representing the serialized transaction that performs the token swap. This transaction can be decoded and processed by the Solana blockchain.

#### lastValidBlockHeight

The block height up to which this transaction is valid. Transactions must be processed within a certain block range to be considered valid.

#### prioritizationFeeLamports

The fee in lamports that prioritizes your transaction in the blockchain's processing queue. Higher fees can lead to faster transaction confirmations.

#### computeUnitLimit

The maximum number of compute units allocated to this transaction. Compute units measure the computational resources required to process the transaction.

#### prioritizationType

Details about how the transaction prioritization fee is handled.

#### computeBudget

- **microLamports:** Fee per micro lamport allocated.
- **estimatedMicroLamports:** The estimated total fee required based on current network conditions.

#### dynamicSlippageReport

Reports any dynamic slippage that occurred during the simulation of the swap. `null` indicates no issues.

#### simulationError

Details any errors encountered during the simulation of the transaction. `null` indicates a successful simulation without errors.

In [4]:
swapTx = base64.b64decode(txRes['swapTransaction'])
pprint(swapTx)

(b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
 b'\x00\x80\x01\x00\x07\x0f(\xb4\xbeL\xe4\xc2\xe72\x96\xc7o"\r\xb2\x8bB\x02\xda'
 b'\xbb\x8c\rS\x94e\xb8I2\xd5\x13\xa3\x10\xeam\xcb8\t\x00\x07|\xda\xbe\xe0'
 b'\x919L\xc8y\x90,:"a\xfcq\xaa\xa9\x8e9f\x80\xda\xc0WB\x96\x05\x93\xd1\x84\x18'
 b'Z\xae\xe7\x8c\x88\x14/JRS\xcb\xe54NxS\xe6\xb2\xe9jT\x84sE\x8a\xfa\xa2\x1d'
 b'\x9aIgU2\xf7\x1a\xb5\xca#\xadkp\\\x04\xe2;\x0e\xff\xa3\x98D\xec\xd1n\xcbz|'
 b'\xbe\x02\xb8+\x0eL\xfcy\x9f\x07\xda\xea\xa60\x11\xb3\xa1u\x04X\xfbi0#'
 b'\xdf\x85\xb45\x96\xb5\x8eY\xd6\xcb\xbe\x05<\xde\x9a\x97\x10\xc3\x9d7'
 b'\x91\xe6\xe17-\x8c+\xc6\x151^\x01\x07`*\xe9*\xfak\xbf6\x0b\xc2\x94c\xca\xb8/'
 b'\xda&0\xc3\xf3\x07\x80\xad\x8f\xf4VTu\xeaE\n/X\xf4\xc2-\xe0k1v\x10\xdf\xb0'
 b'\xb8_\xecW\xfd6\x87

In [5]:
tx1 = VersionedTransaction.from_bytes(swapTx)
pprint(tx1)

VersionedTransaction(
    VersionedTransaction {
        signatures: [
            1111111111111111111111111111111111111111111111111111111111111111,
        ],
        message: V0(
            Message {
                header: MessageHeader {
                    num_required_signatures: 1,
                    num_readonly_signed_accounts: 0,
                    num_readonly_unsigned_accounts: 7,
                },
                account_keys: [
                    3juABNT61D4tFxZ5xod8rwjb2SSWtSe5PSpx4qY1AMHs,
                    8PbAgMN73p1FMaDLqNyoSPeyh5uUTVFDdK5XanufXSZo,
                    B6d64smpU35Zv2QRLHLi4cVXQNVGDpuhcfzKEbpip6ED,
                    BuqEDKUwyAotZuK37V4JYEykZVKY8qo1zKbpfU9gkJMo,
                    DPv6XHnYsghFCcFSRRURKzFPEc17kQYt6vmzwziHCHNr,
                    Dnm6WCV8t1yRfmWeSXw23jwSB7JRe9ZDgif4xY3Hoj2z,
                    E6ZL9MAPGVevNcxiz4CYjJxkDfqdGDzRVtuRaUH6GWTq,
                    G4CD7aqqZZ6QKCNHrc1MPdS9Aw8BWmQ5ZkDd54W6mAEG,
                    11

In [6]:
tx = VersionedTransaction(tx1.message, [KEY])
pprint(tx)

VersionedTransaction(
    VersionedTransaction {
        signatures: [
            E89y8gtZUBjuhUZD3P3jKxVrgeaXdNzNKMoYwqFW7HAseUqnjKN8vuX96emkqPxkwgB2MztaSgy6VgyMtUR5x3S,
        ],
        message: V0(
            Message {
                header: MessageHeader {
                    num_required_signatures: 1,
                    num_readonly_signed_accounts: 0,
                    num_readonly_unsigned_accounts: 7,
                },
                account_keys: [
                    3juABNT61D4tFxZ5xod8rwjb2SSWtSe5PSpx4qY1AMHs,
                    8PbAgMN73p1FMaDLqNyoSPeyh5uUTVFDdK5XanufXSZo,
                    B6d64smpU35Zv2QRLHLi4cVXQNVGDpuhcfzKEbpip6ED,
                    BuqEDKUwyAotZuK37V4JYEykZVKY8qo1zKbpfU9gkJMo,
                    DPv6XHnYsghFCcFSRRURKzFPEc17kQYt6vmzwziHCHNr,
                    Dnm6WCV8t1yRfmWeSXw23jwSB7JRe9ZDgif4xY3Hoj2z,
                    E6ZL9MAPGVevNcxiz4CYjJxkDfqdGDzRVtuRaUH6GWTq,
                    G4CD7aqqZZ6QKCNHrc1MPdS9Aw8BWmQ5ZkDd54W6mAEG,

In [7]:
# Send the raw transaction without skipping preflight checks
response = http_client.send_raw_transaction(bytes(tx), TxOpts(skip_preflight=False))
pprint(response)
txId = response.get('result')
pprint(txId)

if txId:
    print(f"https://solscan.io/tx/{str(txId)}")
else:
    print("Failed to send transaction:", response)

{'id': 1,
 'jsonrpc': '2.0',
 'result': 'E89y8gtZUBjuhUZD3P3jKxVrgeaXdNzNKMoYwqFW7HAseUqnjKN8vuX96emkqPxkwgB2MztaSgy6VgyMtUR5x3S'}
'E89y8gtZUBjuhUZD3P3jKxVrgeaXdNzNKMoYwqFW7HAseUqnjKN8vuX96emkqPxkwgB2MztaSgy6VgyMtUR5x3S'
https://solscan.io/tx/E89y8gtZUBjuhUZD3P3jKxVrgeaXdNzNKMoYwqFW7HAseUqnjKN8vuX96emkqPxkwgB2MztaSgy6VgyMtUR5x3S


In [8]:
def get_token_balances(client, wallet_address):
    """
    Retrieves all token balances for the given wallet address using Solana RPC.

    Args:
        client (solana.rpc.api.Client): Solana RPC client.
        wallet_address (PublicKey): Public key of the wallet.

    Returns:
        List[Dict]: A list of dictionaries containing token mint addresses and their amounts.
    """
    opts = TokenAccountOpts(
        program_id="TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA",
        encoding="jsonParsed"  # You can choose 'jsonParsed' or 'base64'
    )
    token_accounts = client.get_token_accounts_by_owner(wallet_address, opts)
    balances = []
    for account in token_accounts['result']['value']:
        account_info = account['account']['data']['parsed']['info']
        mint = account_info['mint']
        amount = int(account_info['tokenAmount']['amount']) / (10 ** int(account_info['tokenAmount']['decimals']))
        balances.append({'mint': mint, 'amount': amount})
    return balances

def get_token_prices(mint_addresses):
    """
    Retrieves current USD prices for the given list of token mint addresses using Birdeye's Price API.

    Args:
        mint_addresses (List[str]): List of token mint addresses.

    Returns:
        Dict[str, float]: A dictionary mapping mint addresses to their USD prices.
    """
    api_key = d.birdeye  # Ensure your API key is stored securely
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    mint_query = ','.join(mint_addresses)
    response = requests.get(
        f'https://public-api.birdeye.so/defi/price?mints={mint_query}',
        headers=headers
    )
    if response.status_code == 200:
        price_data = response.json()
        return {item['mint']: item['usd_price'] for item in price_data['data']}
    else:
        print("Failed to fetch prices:", response.text)
        return {}

def calculate_usd_values(balances, prices):
    """
    Calculates the USD value for each token balance.

    Args:
        balances (List[Dict]): List of token balances with mint addresses and amounts.
        prices (Dict[str, float]): Dictionary mapping mint addresses to USD prices.

    Returns:
        List[Dict]: List containing token balances along with their USD values.
    """
    detailed_balances = []
    for balance in balances:
        mint = balance['mint']
        amount = balance['amount']
        usd_price = prices.get(mint, 0)
        usd_value = amount * usd_price
        detailed_balances.append({
            'mint': mint,
            'amount': amount,
            'usd_price': usd_price,
            'usd_value': usd_value
        })
    return detailed_balances

def display_wallet_balances():
    """
    Fetches and displays all token balances in the wallet along with their USD values.
    """
    # Initialize Solana RPC client
    http_client = Client("https://api.mainnet-beta.solana.com")

    # Load wallet keypair
    KEY = Keypair.from_base58_string(d.key)
    wallet_address = KEY.pubkey()

    # Fetch token balances
    balances = get_token_balances(http_client, wallet_address)
    print("Token Balances:", balances)

    if not balances:
        print("No token balances found.")
        return

    # Extract mint addresses
    mint_addresses = [balance['mint'] for balance in balances]

    # Fetch token prices
    prices = get_token_prices(mint_addresses)
    print("Token Prices:", prices)

    # Calculate USD values
    detailed_balances = calculate_usd_values(balances, prices)
    pprint(detailed_balances)

    # Optionally, display total portfolio value
    total_usd = sum(item['usd_value'] for item in detailed_balances)
    print(f"Total Portfolio Value: ${total_usd:.2f}")

display_wallet_balances()

Token Balances: [{'mint': 'EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm', 'amount': 0.055248}]
Failed to fetch prices: {"success":false,"message":"Unauthorized"}
Token Prices: {}
[{'amount': 0.055248,
  'mint': 'EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm',
  'usd_price': 0,
  'usd_value': 0.0}]
Total Portfolio Value: $0.00
